In [2]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [10]:
travel_index = pd.read_csv('travel_index_map.csv', index_col=0)
travel_index.head(20)

,latitude,longitude,min travel time,shortest trip,flattened trip time
0,52.500000,13.500000,1073.610024,Michiganseestr. (Berlin) > S Ostkreuz Bhf (Ber...,1073.610024
1,52.500000,13.492857,832.495337,Lückstr./Weitlingstr. (Berlin) > S Ostkreuz Bh...,832.495337
2,52.500000,13.485714,1446.585478,S Nöldnerplatz/Schlichtallee (Berlin) > S Nöld...,1139.604766
3,52.500000,13.478571,821.904976,S Rummelsburg (Berlin) > S+U Warschauer Str. (...,821.904976
4,52.500000,13.471429,1198.938142,S Rummelsburg/Hauptstr. EV (Berlin) > S Rummel...,966.112487
5,52.500000,13.464286,798.590755,Markgrafendamm (Berlin) > S Ostkreuz Bhf (Berl...,798.590755
6,52.500000,13.457143,693.409591,Osthafen (Berlin) > S+U Warschauer Str. (Berli...,693.409591
7,52.500000,13.450000,705.181426,Stralauer Allee (Berlin) > S+U Warschauer Str....,705.181426
8,52.500000,13.442857,629.194537,Falckensteinstr. (Berlin) > U Alexanderplatz [...,629.194537
9,52.500000,13.435714,916.625294,Lausitzer Platz (Berlin) > U Schlesisches Tor ...,916.625294


In [15]:
plt.get_cmap('RdYlGn', 1)

In [30]:
folium.LinearColormap(('g', 'y', 'r'), vmax=100)

In [29]:
m = folium.Map((52.515998, 13.457574), zoom_start=14)
for index, row in travel_index.iterrows():
    mtt = row['min travel time']
    if mtt < 600:
        color = 'green'
    elif mtt < 900:
        color = 'yellow'
    else:
        color = 'red'
    folium.CircleMarker((row['latitude'], row['longitude']), fill_color=color, opacity=1.0, color=color, stroke=True, tooltip='{:0.0f}s ({:0.0f}s): {}'.format(row['flattened trip time'], mtt, row['shortest trip'])).add_to(m)
    
overlay = np.load('./grid_z2.npy')
my_fun = np.vectorize(lambda x: 1 - min(x/2500., 1))
overlay = my_fun(overlay)

[min_lat, min_lon] = np.min(travel_index[['latitude', 'longitude']].to_numpy(), axis=0)
[max_lat, max_lon] = np.max(travel_index[['latitude', 'longitude']].to_numpy(), axis=0)

folium.raster_layers.ImageOverlay(
    image=overlay,
    bounds=[[min_lat, min_lon], [max_lat, max_lon]],
    # colormap=folium.LinearColormap(('g', 'y', 'r'), vmin=0, vmax=100),
    colormap=plt.get_cmap('RdYlGn'),
    opacity=0.7,
    pixelated=False,
    origin='lower'
).add_to(m)

m